### Connecting to Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

### Imports

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from traditional_models import *
from data_processor import *
from deep_models import *
from utils import *

In [ ]:
Result_Export_Path = '/content/gdrive/MyDrive/Fall Detection System/Results/'

## Data Processor

In [ ]:
%%time

data_processor = DatasetProcessor()
print('1.Split Adresses...')
train, test = data_processor.get_file_name('/content/gdrive/MyDrive/Datasets/SisFall_dataset/')

print('2.Extract Features and Labels...')
print('------------------------Train Dataset')
train_dataset = data_processor.datasets_to_nparray(train)
print('------------------------Test Dataset')
test_dataset = data_processor.datasets_to_nparray(test)

print('3.Windowing...')
print('------------------------Train Dataset')
X_train, y_train = data_processor.windowing3d(train_dataset)
print('------------------------Test Dataset')
X_test, y_test = data_processor.windowing3d(test_dataset)

print('4.Normalizing...')
scaler = StandardScaler()
X_train, X_test = data_processor.normalizer(scaler, X_train, X_test)

print('5.Calculate Class Weight...')
class_weight = data_processor.generate_class_weight(y_train)


### Save & Load

In [ ]:
save_data = False
if save_data:
  save_pickle('/content/gdrive/MyDrive/Fall Detection System/Dataset/X_train')
  save_pickle('/content/gdrive/MyDrive/Fall Detection System/Dataset/y_train.pkl')
  save_pickle('/content/gdrive/MyDrive/Fall Detection System/Dataset/X_test.pkl')
  save_pickle('/content/gdrive/MyDrive/Fall Detection System/Dataset/y_test.pkl')

else:
  X_train = load_pickle('/content/gdrive/MyDrive/Fall Detection System/Dataset/X_train.pkl')
  y_train = load_pickle('/content/gdrive/MyDrive/Fall Detection System/Dataset/y_train.pkl')
  X_test = load_pickle('/content/gdrive/MyDrive/Fall Detection System/Dataset/X_test.pkl')
  y_test = load_pickle('/content/gdrive/MyDrive/Fall Detection System/Dataset/y_test.pkl')
  data_processor = DatasetProcessor()
  class_weight = data_processor.generate_class_weight(y_train)


print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

## Traditional Models

#### initialization of Parameters

In [ ]:
model_parameters = {'KNN':{'n_neighbors':[2,4,8,16], 'metric':['euclidean','cosine']},
                    'Random Forest':{'criterion':['gini', 'entropy', 'log_loss']},
                    'Decision Tree':{'criterion':['gini', 'entropy', 'log_loss']},
                    }


tradional_models = Traditional_Models(models = {'Logistic Regression':LogisticRegression(class_weight=class_weight),
                                                'Random Forest':RandomForestClassifier(class_weight=class_weight),
                                                'KNN':KNeighborsClassifier(),
                                                'Decision Tree': DecisionTreeClassifier(class_weight=class_weight)},
                                      model_parameters = model_parameters)

#### Training & Evaluation

In [ ]:
predictions, results = tradional_models.pipeline(X_train,
                                                 y_train,
                                                 X_test,
                                                 y_test,
                                                 class_weight)

#### Save Results

In [ ]:
save_pickle(predictions, Result_Export_Path + 'predictions.pkl')
save_pickle(results, Result_Export_Path + 'results.pkl')

## Deep Models

#### Load Results

In [ ]:
predictions = load_pickle(Result_Export_Path + 'predictions.pkl')
results = load_pickle( Result_Export_Path + 'results.pkl')



### Train & Evaluation

In [ ]:
train_evaluate_deep = Train_Evaluate_Deep(predictions=predictions,
                                          results=results)

#### CNN + MLP

In [ ]:
print('1. Build Model...')
cnn_model = train_evaluate_deep.build_cnn(input_size=X_train.shape[1:])
print('2. Train Model...')
cnn_history = train_evaluate_deep.train_deep_model(cnn_model,
                                                   X_train,
                                                   y_train,
                                                   optimizer=tf.optimizers.SGD(learning_rate=0.001),
                                                   loss_function=keras.losses.BinaryCrossentropy(),
                                                   callbacks=[get_early_stopping(metric='val_PRC', patience=10, mode='max')],
                                                   metrics=get_classifiaction_metrics(),
                                                   class_weight=class_weight,
                                                   epochs=30)
print('3. Evaluation...')
prediction = train_evaluate_deep.evaluate(cnn_model, X_test, y_test, batch_size=128, threshold=0.9, title='CNN + MLP', model_name='CNN + MLP', plot=False)

plot_learning_curves=False
if plot_learning_curves:
  print('4. Plot learning curves...')
  _ = train_evaluate_deep.plot_learning_curves(cnn_history)

#### CNN + LSTM

In [ ]:
print('1. Build Model...')
LSTM_model = train_evaluate_deep.build_lstm(input_size=X_train.shape[1:])
print('2. Train Model...')
LSTM_history = train_evaluate_deep.train_deep_model(LSTM_model,
                                                   X_train,
                                                   y_train,
                                                   optimizer=tf.optimizers.SGD(learning_rate=0.001),
                                                   loss_function=keras.losses.BinaryCrossentropy(),
                                                   callbacks=[get_early_stopping(metric='val_PRC', patience=10, mode='max')],
                                                   metrics=classification_metrics,
                                                   class_weight=class_weight,
                                                   epochs=30)

print('3. Evaluation...')
prediction = train_evaluate_deep.evaluate(LSTM_model, X_test, y_test, batch_size=128, threshold=0.9, title='LSTM + MLP', model_name='LSTM + MLP', plot=False)

plot_learning_curves=False
if plot_learning_curves:
  print('4. Plot learning curves...')
  _ = train_evaluate_deep.plot_learning_curves(LSTM_history)

#### MLP

In [ ]:
print('1. Build Model...')
mlp_model = train_evaluate_deep.build_mlp(input_size=X_train.shape[1:], hidden_layer_size=128, output_size=1)
print('2. Train Model...')
mlp_history = train_evaluate_deep.train_deep_model(mlp_model,
                                                   X_train,
                                                   y_train,
                                                   optimizer=tf.optimizers.SGD(learning_rate=0.001),
                                                   loss_function=keras.losses.BinaryCrossentropy(),
                                                   callbacks=[get_early_stopping(metric='val_PRC', patience=10, mode='max')],
                                                   metrics=classification_metrics,
                                                   class_weight=class_weight,
                                                   epochs=30)

print('3. Evaluation...')
prediction = train_evaluate_deep.evaluate(mlp_model, X_test, y_test, batch_size=128, threshold=0.9, title='MLP', model_name='MLP', plot=False)

plot_learning_curves=False
if plot_learning_curves:
  print('4. Plot learning curves...')
  _ = train_evaluate_deep.plot_learning_curves(mlp_history)

### Conclusion

In [ ]:
convert_report_df(results)